In [1]:
mirna = "UCAGGUCAUGCAUGCAGUCA"

mrna = "GTACGTCACGTAGTACGTACGTCAGTACTGCAGTCAGTCAGTCAGT"

In [2]:
def dna_to_rna(seq):
    """Convert DNA to RNA (T->U)"""
    return seq.upper().replace('T', 'U')

In [5]:
mirna_seq = dna_to_rna(mirna)
mrna_seq = dna_to_rna(mrna)

mrna_seq

'GUACGUCACGUAGUACGUACGUCAGUACUGCAGUCAGUCAGUCAGU'

In [7]:
from src.model.model import TargetNet  # Make sure model.py is in your working directory or in PYTHONPATH
import json
import torch

MODEL_PATH = "pretrained_models/TargetNet.pt"
MODEL_CONFIG_PATH = "config/model/TargetNet.json"

with open (MODEL_CONFIG_PATH, 'r') as f:
    config = json.load(f)

model = TargetNet(
    model_cfg=type('Config', (), config),  # Convert dict to dummy object
    with_esa=True,  # Must match training config
    dropout_rate=0  # Disable for inference
)

# 4. Load weights
model.load_state_dict(torch.load(MODEL_PATH, map_location='cpu'))
model.eval()

TargetNet(
  (stem): Sequential(
    (0): Conv_Layer(
      (relu): ReLU()
      (dropout): Dropout(p=0, inplace=False)
      (conv): Sequential(
        (0): ConstantPad1d(padding=(2, 2), value=0)
        (1): Conv1d(10, 16, kernel_size=(5,), stride=(1,), bias=False)
      )
    )
    (1): Conv_Layer(
      (relu): ReLU()
      (dropout): Dropout(p=0, inplace=False)
      (conv): Sequential(
        (0): ConstantPad1d(padding=(2, 2), value=0)
        (1): Conv1d(16, 16, kernel_size=(5,), stride=(1,), bias=False)
      )
    )
  )
  (stage1): Sequential(
    (0): ResNet_Block(
      (relu): ReLU()
      (dropout): Dropout(p=0, inplace=False)
      (conv1): Sequential(
        (0): ConstantPad1d(padding=(1, 1), value=0)
        (1): Conv1d(16, 16, kernel_size=(3,), stride=(1,), bias=False)
      )
      (conv2): Sequential(
        (0): ConstantPad1d(padding=(1, 1), value=0)
        (1): Conv1d(16, 16, kernel_size=(3,), stride=(1,), bias=False)
      )
    )
  )
  (stage2): Sequential(


In [11]:
# 5. Helper functions
def dna_to_rna(seq):
    return seq.upper().replace('T', 'U')

def reverse_complement(seq):
    comp = {'A':'U', 'U':'A', 'C':'G', 'G':'C'}
    return ''.join([comp.get(c, 'N') for c in seq[::-1]])

# 6. Sample sequences (replace with yours)
miRNA_dna = "TGAGGTAGATCTCACTGTAGA"  # Example miRNA 
mRNA_dna = "ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG"  # Your 51nt

# 7. Preprocess sequences
miRNA = dna_to_rna(miRNA_dna)
mRNA_rc = reverse_complement(dna_to_rna(mRNA_dna))  # Reverse complement

In [12]:
from Bio import pairwise2


In [15]:
# Create score matrix (from original code)
score_matrix = {}
for c1 in 'ACGU':
    for c2 in 'ACGU':
        if (c1,c2) in [('A','U'),('U','A'),('G','C'),('C','G'),('U','G'),('G','U')]:
            score_matrix[(c1,c2)] = 1
        else:
            score_matrix[(c1,c2)] = 0

alignment = pairwise2.align.globaldx(
    miRNA[:10], mRNA_rc[5:15],
    score_matrix, one_alignment_only=True
)[0]

In [16]:
import numpy as np

In [17]:

def one_hot(seq):
    mapping = {'A':[1,0,0,0], 'U':[0,1,0,0], 
                'C':[0,0,1,0], 'G':[0,0,0,1]}
    encoded = np.zeros((4,50), dtype=np.float32)
    for i in range(50):
        encoded[:,i] = mapping.get(seq[i].upper() if i<len(seq) else 'N', [0,0,0,0])
    return encoded

# Build 10x50 tensor (channels:4miRNA +4mRNA +1match +1position)
mirna_oh = one_hot(miRNA.ljust(50, 'N'))
mrna_oh = one_hot(mRNA_rc.ljust(50, 'N'))

In [19]:
# ESA Features (pad alignment to 50nt)
align_mi_pad = alignment.seqA.ljust(50, 'N')[:50]
align_mrna_pad = alignment.seqB.ljust(50, 'N')[:50]
match_feature = np.array([1 if m==w else 0 for m,w in zip(align_mi_pad, align_mrna_pad)])
positional_feature = np.linspace(0, alignment.score/10, 50)


In [25]:
alignment[2]

7.0